In [ ]:
import random
import sys

from loguru import logger

from GraphOps import GraphOps
from Graph import Graph
from Node import Node
from SchemaType import SchemaType

In [ ]:
logger.remove()
logger.add(
    sys.stderr,
    enqueue=True,
    backtrace=True,
    diagnose=True,
    colorize=True,
    level="INFO",
)

In [ ]:
def generate_random_graph(size: int) -> Graph:
    g = Graph()
    for i in range(size):
        n = Node(
            random.randint(1, 10) / 10,
            SchemaType.first() + (i % len(SchemaType)),
            f"n{i}",
        )
        if random.random() > 0.5:
            n.value = 1.0
        g.add_node(n)
        if i > 1:
            g.add_edge(n, g.nodes[-2], weight=random.random())
            g.add_edge(random.choice(g.nodes[:-2]), n, weight=random.random())

    return g


In [ ]:
g = generate_random_graph(30)

logger.info(str(g))

graph_ops = GraphOps(
    g,
    spread_schema=SchemaType,
    decay_factor=0.9,
    threshold=0.3,
    max_steps=10,
)

logger.info("Starting the graph operations")
graph_ops.create_priority_queue()
logger.info("Finished traversing the graph")
graph_ops.activate_priority_queue()
logger.info("Finished the graph operations")

logger.info(str(g))
